In [ ]:
# BLOQUE 1: BIBLIOTECAS Y ELECCIÓN DEL DATASET

# Bibliotecas necesarias.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Elección, carga y estudio del dataset "smartphones.csv".

url = "https://raw.githubusercontent.com/alvarovacch/DatasetSmartphones/refs/heads/main/smartphones.csv"
dataset = pd.read_csv(url)
dataset

In [ ]:
# BLOQUE 2.1: PREPARACIÓN DEL DATASET = PREPROCESADO DE DATOS

# Eliminación de la columna "Smartphone".
# Se elimina dicha columna al contener valores únicos en cada fila los cuales, no aportan información relevante para la predicción del "Final Price".

dataset_filtrado1 = dataset.drop(['Smartphone'], axis=1)

# Eliminación de aquellas marcas que presentan un único móvil.
# Dicha eliminación se lleva a cabo con el fin mejorar la predicción de la relación entre las diferentes variables del dataset con el "Final Price".

conteo_brand1 = dataset_filtrado1['Brand'].value_counts()
print(conteo_brand1)

valores_a_eliminar1 = conteo_brand1[conteo_brand1 == 1].index
dataset_filtrado2 = dataset_filtrado1[~dataset_filtrado1['Brand'].isin(valores_a_eliminar1)] # Eliminación de las filas que contienen las marcas citadas.

conteo_brand2 = dataset_filtrado2['Brand'].value_counts()
print(conteo_brand2)

# Eliminación de aquellos modelos que presentan un único móvil.
# Dicha eliminación se lleva a cabo con el fin mejorar la predicción de la relación entre las diferentes variables del dataset con el "Final Price".

conteo_model1 = dataset_filtrado2['Model'].value_counts()
print(conteo_model1)

valores_a_eliminar2 = conteo_model1[conteo_model1 == 1].index
dataset_filtrado3 = dataset_filtrado2[~dataset_filtrado2['Model'].isin(valores_a_eliminar2)] # Eliminación de las filas que contienen los modelos citados.

conteo_model2 = dataset_filtrado3['Model'].value_counts()
print(conteo_model2)

# Eliminación de aquellas capacidades que presentan un único móvil.
# Dicha eliminación se lleva a cabo con el fin mejorar la predicción de la relación entre las diferentes variables del dataset con el "Final Price".

conteo_Storage1 = dataset_filtrado3['Storage'].value_counts()
print(conteo_Storage1)

valores_a_eliminar3 = conteo_Storage1[conteo_Storage1 == 1].index
dataset_filtrado4 = dataset_filtrado3[~dataset_filtrado3['Storage'].isin(valores_a_eliminar3)] # Supresión de las filas que tienen las capacidades citadas.

conteo_Storage2 = dataset_filtrado4['Storage'].value_counts()
print(conteo_Storage2)

# Eliminación de aquellos colores que presentan un único móvil.
# Dicha eliminación se lleva a cabo con el fin mejorar la predicción de la relación entre las diferentes variables del dataset con el "Final Price".

conteo_Color1 = dataset_filtrado4['Color'].value_counts()
print(conteo_Color1)

valores_a_eliminar4 = conteo_Color1[conteo_Color1 == 1].index
dataset_filtrado5 = dataset_filtrado4[~dataset_filtrado4['Color'].isin(valores_a_eliminar4)] # Eliminación de las filas que contienen los colores citados.

conteo_Color2 = dataset_filtrado5['Color'].value_counts()
print(conteo_Color2)

# Sustitución de los valores "NaN" de la columna "RAM" por la mediana.
# Dicha sustitución se lleva a cabo ya que la distribución de "RAM" puede estar sesgada (4GB - 8GB).
# La mediana es menos sensible a los valores extremos.

print("Valores NaN:", dataset_filtrado5.isnull().sum())

conteo_RAM1 = dataset_filtrado5['RAM'].value_counts()
print(conteo_RAM1) # Muestra el número de teléfonos que tiene cada cantidad concreta de "RAM" (útil para conocer si una distribución está sesgada o no).

dataset_filtrado5.loc[:, 'RAM'] = dataset_filtrado5['RAM'].fillna(dataset_filtrado5['RAM'].median()) # (:,'RAM') Selecciona cada valor de la columna "RAM".

print("Valores NaN:", dataset_filtrado5.isnull().sum())

# Eliminación de los valores "NaN" de la columna "Storage".
# Dicha eliminación no afecta significativamente a la relación entre las diferentes variables del dataset con el "Final Price" al ser escasos (25).

dataset_filtrado5 = dataset_filtrado5.dropna(subset=['Storage'])
print("Valores NaN:", dataset_filtrado5.isnull().sum())


In [14]:
# BLOQUE 2.2: PREPARACIÓN DEL DATASET = PREPROCESADO DE DATOS

# Conversión de las variables categóricas a numéricas (toman 0 o 1 como valores).
# Dicha conversión se lleva a cabo para poder generar la matriz de correlación y visualizar de tal forma, la relación de cada variable con "Final Price".
# Gracias al "drop_first=False" se mantienen todas las columnas generadas.

dataset_filtrado6 = pd.get_dummies(dataset_filtrado5, columns=['Brand', 'Model', 'Color', 'Free'], drop_first=False)
dataset_filtrado6

# Cálculo de la matriz de correlación.
matriz_correlacion = dataset_filtrado6.corr()
matriz_correlacion

# Filtrado de la matriz de correlación por correlación significativa con "Final Price" y por valor de correlación (mayor que 0,15).
# Dichas filtraciones se llevan a cabo ya que simplifican el modelo y mejoran la interpretabilidad (sin perder las relaciones más importantes).
matriz_filtrada1 = matriz_correlacion['Final Price']
matriz_filtrada2 = matriz_filtrada1[abs(matriz_filtrada1) > 0.15] # Considera tanto los valores positivos como negativos (magnitud mayor que 0,15).

print ("Correlaciones con 'Final Price' mayores a 0.15 (en valor absoluto):")
print(matriz_filtrada2)


Correlaciones con 'Final Price' mayores a 0.15 (en valor absoluto):
RAM                     0.442014
Storage                 0.699279
Final Price             1.000000
Brand_Apple             0.383505
Brand_Samsung           0.215502
Brand_Xiaomi           -0.222800
Model_Galaxy S23        0.290307
Model_Galaxy Z Fold4    0.280856
Model_iPhone 13         0.344734
Model_iPhone 14         0.441714
Color_Purple            0.158280
Name: Final Price, dtype: float64


In [ ]:
# BLOQUE 3.1. SCIKIT - LEARN (REGRESIÓN LINEAL - MÍNIMOS CUADRADOS)

# División de las variables en variables predictoras (X (todas las columnas salvo 'Final Price')) y variable objetivo (Y(únicamente 'Final Price')).
X = dataset_filtrado6[matriz_filtrada2.index.drop('Final Price')]
Y = dataset_filtrado6['Final Price']

# División de los datos del dataset en "Train" (80%) y "Test" (20%).
# Se lleva a cabo dicha división en subconjuntos de entrenamiento (80%) y prueba (20%).
# Gracias al "random_state=42" la división de los datos es siempre la misma cada vez que se ejecuta el código.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Creación del modelo de regresión lineal.
modelo_de_regresion_lineal = LinearRegression()

# Entrenamiento del modelo de regresión lineal.
entrenamiento_modelo_de_regresion_lineal = modelo_de_regresion_lineal.fit(X_train, Y_train)

print("Características del entrenamiento:")
print(X_train)

print("Valores reales del entrenamiento:")
print(Y_train)

# Prueba del modelo de regresión lineal.
prueba_modelo_de_regresion_lineal = modelo_de_regresion_lineal.predict(X_test)

print("Características de la prueba:")
print(X_test)

print("Valores reales de la prueba:")
print(Y_test)

# Evaluación del modelo de regresión lineal.
# Se lleva a cabo dicha evaluación con el fin de medir como de buenas son las predicciones realizadas.
# Utilizamos el "RMSE" en lugar del "MSE" ya que de tal forma; es posible interpretar los resultados al tener todos los valores en las mismas unidades.

print("Evaluación del modelo de regresión lineal (SCIKIT - LEARN):")
print(f"Coeficiente de determinación (R²): {r2_score(Y_test, prueba_modelo_de_regresion_lineal)}")
print(f"Raíz del error cuadrático medio (RMSE): {mean_squared_error(Y_test, prueba_modelo_de_regresion_lineal, squared=False):.2f}")

In [ ]:
# BLOQUE 3.2: ALGORITMO IMPLEMENTADO (REGRESIÓN LINEAL - MÍNIMOS CUADRADOS)

# Empleamos dicho algoritmo con el fin de predecir el precio de los teléfonos en función de sus respectivas características ("Brand", "Model", "RAM"...).
# Para ello, calcularemos aquellas betas (describen la relación entre las variables predictoras y la variable objetivo) que minimicen el error cuadrático.
# Error cuadrático = Diferencia entre las predicciones y los valores reales = Fórmula de los Mínimos Cuadrados Ordinarios (OLS): B = ((X^T X)^-1) X^T Y.
# Finalmente, realizaremos las predicciones para calcular el precio de los teléfonos (al disponer de las betas y de la matriz de variables predictoras).
# Fórmula de las predicciones:  Y = X B

# Elementos de la "Fórmula de los Mínimos Cuadrados Ordinarios (OLS)":
#   - B = Vector de coeficientes = Vector de betas (B_0;B_1;B_2;...;B_n).
#   - X = Matriz de las variables predictoras.
#   - Y = Vector de los valores reales.
#   - X^T = Transpuesta de la matriz de variables predictoras.
#   - (X^T X)^-1 = Inversa de la matriz "X^T X".

# Creación y agregación del intercepto (B_0) la matriz de variables predictoras.
# Se lleva a cabo dicha creación y agregación con el fin de representar el intercepto (columna repleta de 1) en el modelo de regresión lineal.
# Dicho intercepto se agrega a la matriz de variables predictoras ya que no depende de dichas variables y es necesario para minimizar el error cuadrático.

beta_0_train = np.ones((X_train.shape[0], 1)) # Crea una matriz con tantas filas como tenga la matriz de variables predictoras y con 1 columna.

print("B_0 (Train):")
print(beta_0_train)

X_train_y_beta_0_train = np.hstack([beta_0_train, X_train]) # Agrega la columna creada (repleta de 1) a la matriz de variables predictoras.

print("B_0 + Matriz de variables predictoras (B_0 + X (Train)):")
print(X_train_y_beta_0_train)

beta_0_test = np.ones((X_test.shape[0], 1)) # Crea una matriz con tantas filas como tenga la matriz de variables predictoras y con 1 columna.

print("B_0 (Test):")
print(beta_0_test)

X_test_y_beta_0_test = np.hstack([beta_0_test, X_test]) # Agrega la columna creada (repleta de 1) a la matriz de variables predictoras.

print("B_0 + Matriz de variables predictoras (B_0 + X (Test)):")
print(X_test_y_beta_0_test)

In [ ]:
# BLOQUE 3.3: ALGORITMO IMPLEMENTADO (REGRESIÓN LINEAL - MÍNIMOS CUADRADOS)

# Cálculo de las betas mediante la fórmula del "OLS".
# Se lleva a cabo dicho procedimiento a través del cálculo de cada una de las partes que componen la fórmula del "OLS".
# Previamente al inicio de dicho procedimiento, es necesario transformar todas las columnas a tipo numérico para poder realizar los cálculos de matrices.

X_train_y_beta_0_train = X_train_y_beta_0_train.astype(float)

X_train_transpuesta = X_train_y_beta_0_train.T # X^T
X_train_transpuesta_y_X_train_y_beta_0_train = X_train_transpuesta.dot(X_train_y_beta_0_train) # X^T * X (Producto matricial).
X_train_transpuesta_y_X_train_y_beta_0_train_inversa = np.linalg.inv(X_train_transpuesta_y_X_train_y_beta_0_train) # (X^T * X)^-1 (Inversa).
X_train_transpuesta_y_Y_train = X_train_transpuesta.dot(Y_train) # X^T * Y (Producto matricial).

Betas = X_train_transpuesta_y_X_train_y_beta_0_train_inversa.dot(X_train_transpuesta_y_Y_train) # B = ((X^T * X)^-1) * (X^T * Y)

print("Betas Calculadas:")
print(Betas) # [B_0;B_1;B_2;...;B_n]. Cada valor representa cuanto aumenta / disminuye el "Final Price" cuando aumenta una característica en 1 unidad.

# Realización de las predicciones (Y) mediante su propia fórmula.
# Se lleva a cabo dicho procedimiento con el fin de predecir el precio de los teléfonos en función de sus características.

predicciones = X_test_y_beta_0_test.dot(Betas) # Y = X * B (Producto matricial)

print("Predicciones:")
print(predicciones)

# Evaluación del modelo de regresión lineal.
# Se lleva a cabo dicha evaluación con el fin de medir como de buenas son las predicciones realizadas.
# Utilizamos el "RMSE" en lugar del "MSE" ya que de tal forma; es posible interpretar los resultados al tener todos los valores en las mismas unidades.

print("Evaluación del modelo de regresión lineal (MÍNIMOS CUADRADOS):")
print(f"Coeficiente de determinación (R²): {r2_score(Y_test, predicciones)}")
print(f"Raíz del error cuadrático medio (RMSE): {mean_squared_error(Y_test, predicciones, squared=False):.2f}")

CONCLUSIONES:

    - DESEMPEÑO DEL MODELO:

Tanto el modelo de "Scikit Learn" como el modelo de "Mínimos Cuadrados" obtuvieron un R² de 0.7694 y un RMSE de 209,2.

Un R² igual a 76,94% indica que el 76,94% de los cambios en los precios finales de los teléfonos pueden explicarse con las variables predictoras incluidas en el modelo ("Brand", "Model", "RAM"...) lo cual, es un buen indicador de la calidad del modelo ya que está bastante cerca del valor ideal (100%).

Asimismo, un RMSE  igual a 209,2 indica que el modelo puede predecir el precio final de los teléfonos con un error promedio de 209,2 unidades monetarias (posibles sobreestimaciones / subestimaciones).

    - RESULTADOS IDÉNTICOS EN AMBOS MÉTODOS:

Como se ha mencionado anteriormente, ambos modelos ("Scikit Learn" y "Mínimos Cuadrados") han obtenido los mismos valores de R² y RMSE.

Dicho suceso ocurre ya que el modelo "Scikit Learn" utiliza internamente el método de "Mínimos Cuadrados" lo cual, garantiza que dichos resultados sean idénticos y confirma que el modelo ha sido implementado correctamente.

    - CARACTERÍSTICAS MÁS RELEVANTES:

Analizando los resultados obtenidos, se verifica que variables como "RAM" y "Storage" tienen una gran influencia en el precio de los teléfonos al poseer una correlación significativa (0,442014 y 0,699279 respectivamente) con la variable objetivo ("Final Price").

Además, variables como "Brand_Apple", "Model_Iphone_13" y "Model_Iphone_14" muestran del mismo modo, una correlación considerable (0,383505; 0,344734 y 0,441714 respectivamente) con la variable objetivo lo cual, indica que son factores clave en la determinación del precio final de los teléfonos.

Sin embargo, existen variables como "Color_Purple" que presentan una correlación con la variable objetivo más baja (0,158280) lo cual, sugiere que tiene un impacto mucho menor en el precio final de los teléfonos (en comparación con las variables más influyentes citadas).

    - POSIBLES MEJORAS:

Examinando los resultados obtenidos, podría ser útil incluir más características / variables relevantes (fecha de lanzamiento del teléfono, duración de la batería, tipo de procesador) que ayuden a mejorar el ajuste del modelo (reduciendo el RMSE y aumentando el R²).

Asimismo, realizar una normalización / estandarización de las variables podría ser beneficioso ya que garantizaría una escala uniforme de todas las variables lo cual, podría mejorar la precisión del modelo y evitar que las variables con rangos más grandes dominen la predicción.



